Using NLTK to process and lemmatize ingredients

In [73]:
import nltk

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [79]:
import nltk

nltk.download('punkt')  
nltk.download('averaged_perceptron_tagger')  
nltk.download('wordnet')  
nltk.download('omw-1.4')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [80]:
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import ast
import pandas as pd
import html

lemmatizer = WordNetLemmatizer()

def convert_pos_tag_to_wordnet(tag):
    tag = tag.upper()
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    return None 

def lemmatize_word(word, tag):
    wordnet_tag = convert_pos_tag_to_wordnet(tag)
    if wordnet_tag:
        return lemmatizer.lemmatize(word, wordnet_tag)
    return word  

def clean_ingredient_text(ingredient):
    units = [
        "cup", "cups", "teaspoon", "tsp", "tablespoon", "tbsp", "oz", "ounce", "ounces",
        "gram", "grams", "kg", "ml", "liter", "liters", "lbs", "pound", "pounds", "pack",
        "pcs", "pieces", "slices", "dash", "pinch", "lb", "can"
    ]
    
    unwanted_phrases = [
        "taste", "canned", "package", "or", "drain", "rinse", "with", "is", "best",
        "to", "cooked", "chopped", "cook", "chop", "not", "evaporate", "cut", "into", "tablespoons",
        "teaspoons", "but", "I", "prefer", "bite", "sized", "cubes", "and", "cans", "at", "room",
        "temperature", "pkge", "in", "half", "thinly", "diced", "dice", "slice", "sliced",
        "strip", "strips", "large", "small", "uncooked", "old", "fashion"
    ]

    # Remove unwanted words and collapse multiple spaces
    ingredient = re.sub(r'\b\d+(\.\d+)?\b', '', ingredient)  
    ingredient = re.sub(r'\b(?:' + '|'.join(units) + r')\b', '', ingredient, flags=re.IGNORECASE)  
    ingredient = re.sub(r'\b(?:' + '|'.join(unwanted_phrases) + r')\b', '', ingredient, flags=re.IGNORECASE) 
    ingredient = re.sub(r'[^\w\s]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    return ingredient

def process_ingredient(ingredient):
    cleaned_ingredient = clean_ingredient_text(ingredient)
    tokens = word_tokenize(cleaned_ingredient)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatize_word(word, tag) for word, tag in tagged_tokens]
    
    return " ".join(lemmatized_tokens)

def process_ingredient_list(ingredient_list):
    ingredients = [ingredient.strip() for ingredient in ingredient_list.split(',')]
    processed_ingredients = [process_ingredient(ingredient) for ingredient in ingredients]
    
    return ', '.join(processed_ingredients)


In [81]:
converters = {
    column: html.unescape 
    for column in ("name", "description")
} | {
    column: lambda value: tuple(map(html.unescape, ast.literal_eval(value)))
    for column in ("ingredients", "ingredients_raw_str", "search_terms", "steps")
} | {
    "tags": lambda value: frozenset(map(html.unescape, ast.literal_eval(value)))
}

df = pd.read_csv("Data/recipes_food_com_revised.csv", converters = converters)
df.head(10)


,ID,Name,Description,IngredientsExtracted,IngredientsRaw,serving_size,Servings,Instructions,Tags,SearchTerms,TotalTime,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","('water', 'grits', 'salt', 'cheddar cheese', '...","('4 cups water', '1 cup uncooked old ...",1 (155 g),8,"('I a sauce pan, bring water to a boil; slowly...","frozenset({'stove-top', 'main-ingredient', 'se...","('low-calorie', 'side', 'vegetarian', 'diabeti...",65,144.8,6.7,3.3,14.9,382.7,15.7,0.9,0.2,5.0
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","('onion', 'red bell pepper', 'garlic cloves', ...","('1 medium onion, chopped coarse ', '1 ...",1 (366 g),4,"('In a food processor, pulse the onion, red pe...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner', 'shrimp')",45,756.5,28.9,9.0,191.8,2094.9,82.3,3.9,9.7,39.2
2,41090,black-and-white bean salad,NaN,"('white beans', 'canned black beans', 'tomatoe...","('1 cup canned white beans, rinsed and dra...",1 (807 g),1,"('In a large bowl, combine beans, tomato, onio...","frozenset({'technique', 'main-ingredient', 'pr...","('salad', 'vegan', 'side', 'vegetarian', 'dinn...",5,159.0,1.6,0.3,0.0,318.3,28.3,8.3,2.1,9.1
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"('zucchini', 'yellow squash', 'diced tomatoes'...","('2 zucchini, sliced ', '2 small ye...",1 (244 g),4,('Put all ingredients in the crock pot and coo...,"frozenset({'beginner-cook', 'main-ingredient',...","('side', 'vegetarian', 'italian')",370,47.1,0.4,0.1,0.0,19.1,10.4,2.7,4.6,2.6
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"('beef stew meat', 'flour', 'salt', 'allspice'...","('3 lbs beef stew meat', '3 tablespoons...",1 (358 g),8,"('Preheat oven to 350°F.', ""Cut beef into 1 in...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner',)",165,691.1,45.8,17.7,177.1,503.6,14.4,1.6,6.8,48.2
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"('slivered almonds', 'cider vinegar', 'sugar',...","('12 ounces slivered almonds', '1/4 cup ...",1 (832 g),1,('Preheat oven to 375°F Place almonds in sing...,"frozenset({'course', 'preparation', 'candy', '...","('dessert',)",80,2051.9,96.9,7.3,0.0,2668.3,280.2,24.6,243.1,41.9
6,232076,Retro Chicken & Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","('chicken breasts', 'green onion', 'red bell p...","('4 cups roasted chopped chicken breasts',...",1 (85 g),6,"('In large bowl, combine chicken, green onion,...","frozenset({'30-minutes-or-less', 'main-ingredi...","('casserole', 'dinner', 'chicken')",25,515.5,39.0,15.4,49.0,649.3,30.6,2.4,4.0,13.3
7,232083,Asparagus Omelette Wraps,"These wraps make a lovely breakfast, light lun...","('eggs', 'milk', 'fresh sage', 'fresh thyme', ...","('8 eggs', '1/2 cup milk', '1 tabl...",1 (499 g),4,"('Beat the eggs in a bowl. Add the milk, sage,...","frozenset({'cuisine', 'eggs', 'asparagus', 'ma...","('gluten-free', 'breakfast')",40,305.4,18.2,5.0,376.3,207.8,17.8,7.5,5.1,22.4
8,79222,Potato-Crab Chowder,Soup for the soul!,"('butter', 'onion', 'garlic', 'potatoes', 'flo...","('2 tablespoons butter', '1 medium o...",1 (362 g),6,('Saute onion& garlic in melted butter in larg...,"frozenset({'healthy', 'stove-top', 'main-ingre...","('low-calorie', 'healthy', 'low-sodium', 'low-...",45,274.2,8.2,4.6,50.3,446.4,36.8,3.1,3.8,16.0
9,393638,Sweet and Simple Sloppy Joes,Easy and kid-friendly recipe that I always hav...,"('lean ground beef', 'ketchup', 'heinz chili s...","('1 lb lean ground beef', '1/2 cup ke...",1 (103 g),4,('Brown ground beef with onion powder in a fr...,"frozenset({'30-minutes-or-less', 'beginner-coo...","('low-carb', 'lunch')",30,243.7,11.5,4.6,73.7,537.7,10.6,0.7,8.8,23.5


In [82]:
df['NLP_Ingredients'] = df['IngredientsExtracted'].apply(process_ingredient_list)
df.head(10)

,ID,Name,Description,IngredientsExtracted,IngredientsRaw,serving_size,Servings,Instructions,Tags,SearchTerms,...,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,NLP_Ingredients
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","('water', 'grits', 'salt', 'cheddar cheese', '...","('4 cups water', '1 cup uncooked old ...",1 (155 g),8,"('I a sauce pan, bring water to a boil; slowly...","frozenset({'stove-top', 'main-ingredient', 'se...","('low-calorie', 'side', 'vegetarian', 'diabeti...",...,144.8,6.7,3.3,14.9,382.7,15.7,0.9,0.2,5.0,"water, grit, salt, cheddar cheese, garlic, oli..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","('onion', 'red bell pepper', 'garlic cloves', ...","('1 medium onion, chopped coarse ', '1 ...",1 (366 g),4,"('In a food processor, pulse the onion, red pe...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner', 'shrimp')",...,756.5,28.9,9.0,191.8,2094.9,82.3,3.9,9.7,39.2,"onion, red bell pepper, garlic clove, shrimp, ..."
2,41090,black-and-white bean salad,NaN,"('white beans', 'canned black beans', 'tomatoe...","('1 cup canned white beans, rinsed and dra...",1 (807 g),1,"('In a large bowl, combine beans, tomato, onio...","frozenset({'technique', 'main-ingredient', 'pr...","('salad', 'vegan', 'side', 'vegetarian', 'dinn...",...,159.0,1.6,0.3,0.0,318.3,28.3,8.3,2.1,9.1,"white bean, black bean, tomato, onion, celery,..."
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"('zucchini', 'yellow squash', 'diced tomatoes'...","('2 zucchini, sliced ', '2 small ye...",1 (244 g),4,('Put all ingredients in the crock pot and coo...,"frozenset({'beginner-cook', 'main-ingredient',...","('side', 'vegetarian', 'italian')",...,47.1,0.4,0.1,0.0,19.1,10.4,2.7,4.6,2.6,"zucchini, yellow squash, tomato, onion, garlic..."
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"('beef stew meat', 'flour', 'salt', 'allspice'...","('3 lbs beef stew meat', '3 tablespoons...",1 (358 g),8,"('Preheat oven to 350°F.', ""Cut beef into 1 in...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner',)",...,691.1,45.8,17.7,177.1,503.6,14.4,1.6,6.8,48.2,"beef stew meat, flour, salt, allspice, cinnamo..."
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"('slivered almonds', 'cider vinegar', 'sugar',...","('12 ounces slivered almonds', '1/4 cup ...",1 (832 g),1,('Preheat oven to 375°F Place almonds in sing...,"frozenset({'course', 'preparation', 'candy', '...","('dessert',)",...,2051.9,96.9,7.3,0.0,2668.3,280.2,24.6,243.1,41.9,"sliver almond, cider vinegar, sugar, sugar, sa..."
6,232076,Retro Chicken & Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","('chicken breasts', 'green onion', 'red bell p...","('4 cups roasted chopped chicken breasts',...",1 (85 g),6,"('In large bowl, combine chicken, green onion,...","frozenset({'30-minutes-or-less', 'main-ingredi...","('casserole', 'dinner', 'chicken')",...,515.5,39.0,15.4,49.0,649.3,30.6,2.4,4.0,13.3,"chicken breast, green onion, red bell pepper, ..."
7,232083,Asparagus Omelette Wraps,"These wraps make a lovely breakfast, light lun...","('eggs', 'milk', 'fresh sage', 'fresh thyme', ...","('8 eggs', '1/2 cup milk', '1 tabl...",1 (499 g),4,"('Beat the eggs in a bowl. Add the milk, sage,...","frozenset({'cuisine', 'eggs', 'asparagus', 'ma...","('gluten-free', 'breakfast')",...,305.4,18.2,5.0,376.3,207.8,17.8,7.5,5.1,22.4,"egg, milk, fresh sage, fresh thyme, garlic clo..."
8,79222,Potato-Crab Chowder,Soup for the soul!,"('butter', 'onion', 'garlic', 'potatoes', 'flo...","('2 tablespoons butter', '1 medium o...",1 (362 g),6,('Saute onion& garlic in melted butter in larg...,"frozenset({'healthy', 'stove-top', 'main-ingre...","('low-calorie', 'healthy', 'low-sodium', 'low-...",...,274.2,8

In [83]:
def remove_duplicates(df, column_name):
    def process_ingredient_string(ingredient):
        try:
            # Attempt to parse the ingredient as a tuple or list
            parsed_ingredient = ast.literal_eval(ingredient)
            if isinstance(parsed_ingredient, (tuple, list)):
                # Remove duplicates and return as a sorted list
                return tuple(sorted(set(parsed_ingredient)))
        except (ValueError, SyntaxError):
            # Handle as a plain string if parsing fails
            return tuple(sorted(set([item.strip() for item in ingredient.split(',') if item.strip()])))
        
    df[column_name] = df[column_name].apply(process_ingredient_string)
    return df

In [84]:
df = remove_duplicates(df, 'NLP_Ingredients')
df.head(10)

,ID,Name,Description,IngredientsExtracted,IngredientsRaw,serving_size,Servings,Instructions,Tags,SearchTerms,...,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,NLP_Ingredients
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","('water', 'grits', 'salt', 'cheddar cheese', '...","('4 cups water', '1 cup uncooked old ...",1 (155 g),8,"('I a sauce pan, bring water to a boil; slowly...","frozenset({'stove-top', 'main-ingredient', 'se...","('low-calorie', 'side', 'vegetarian', 'diabeti...",...,144.8,6.7,3.3,14.9,382.7,15.7,0.9,0.2,5.0,"(cheddar cheese, garlic, grit, olive oil, salt..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","('onion', 'red bell pepper', 'garlic cloves', ...","('1 medium onion, chopped coarse ', '1 ...",1 (366 g),4,"('In a food processor, pulse the onion, red pe...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner', 'shrimp')",...,756.5,28.9,9.0,191.8,2094.9,82.3,3.9,9.7,39.2,"(andouille sausage, bay leaf, clam juice, fres..."
2,41090,black-and-white bean salad,NaN,"('white beans', 'canned black beans', 'tomatoe...","('1 cup canned white beans, rinsed and dra...",1 (807 g),1,"('In a large bowl, combine beans, tomato, onio...","frozenset({'technique', 'main-ingredient', 'pr...","('salad', 'vegan', 'side', 'vegetarian', 'dinn...",...,159.0,1.6,0.3,0.0,318.3,28.3,8.3,2.1,9.1,"(black bean, black pepper, celery, italian par..."
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"('zucchini', 'yellow squash', 'diced tomatoes'...","('2 zucchini, sliced ', '2 small ye...",1 (244 g),4,('Put all ingredients in the crock pot and coo...,"frozenset({'beginner-cook', 'main-ingredient',...","('side', 'vegetarian', 'italian')",...,47.1,0.4,0.1,0.0,19.1,10.4,2.7,4.6,2.6,"(garlic, green bell pepper, italian seasoning,..."
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"('beef stew meat', 'flour', 'salt', 'allspice'...","('3 lbs beef stew meat', '3 tablespoons...",1 (358 g),8,"('Preheat oven to 350°F.', ""Cut beef into 1 in...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner',)",...,691.1,45.8,17.7,177.1,503.6,14.4,1.6,6.8,48.2,"(allspice, beef stew meat, beef stock, black p..."
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"('slivered almonds', 'cider vinegar', 'sugar',...","('12 ounces slivered almonds', '1/4 cup ...",1 (832 g),1,('Preheat oven to 375°F Place almonds in sing...,"frozenset({'course', 'preparation', 'candy', '...","('dessert',)",...,2051.9,96.9,7.3,0.0,2668.3,280.2,24.6,243.1,41.9,"(cayenne pepper, cider vinegar, ground coriand..."
6,232076,Retro Chicken & Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","('chicken breasts', 'green onion', 'red bell p...","('4 cups roasted chopped chicken breasts',...",1 (85 g),6,"('In large bowl, combine chicken, green onion,...","frozenset({'30-minutes-or-less', 'main-ingredi...","('casserole', 'dinner', 'chicken')",...,515.5,39.0,15.4,49.0,649.3,30.6,2.4,4.0,13.3,"(cheddar cheese, chicken breast, dijon mustard..."
7,232083,Asparagus Omelette Wraps,"These wraps make a lovely breakfast, light lun...","('eggs', 'milk', 'fresh sage', 'fresh thyme', ...","('8 eggs', '1/2 cup milk', '1 tabl...",1 (499 g),4,"('Beat the eggs in a bowl. Add the milk, sage,...","frozenset({'cuisine', 'eggs', 'asparagus', 'ma...","('gluten-free', 'breakfast')",...,305.4,18.2,5.0,376.3,207.8,17.8,7.5,5.1,22.4,"(asparagus, egg, extra virgin olive oil, fresh..."
8,79222,Potato-Crab Chowder,Soup for the soul!,"('butter', 'onion', 'garlic', 'potatoes', 'flo...","('2 tablespoons butter', '1 medium o...",1 (362 g),6,('Saute onion& garlic in melted butter in larg...,"frozenset({'healthy', 'stove-top', 'main-ingre...","('low-calorie', 'healthy', 'low-sodium', 'low-...",...,274.2,8

In [86]:
def remove_empty(ingredient):
    try:
        parsed_ingredient = ast.literal_eval(ingredient)
        if isinstance(parsed_ingredient, (tuple, list)):
            return tuple(sorted(item for item in parsed_ingredient if item))
        else:
            raise ValueError
    except (ValueError, SyntaxError):
        if isinstance(ingredient, str):
            items = [item.strip() for item in ingredient.split(',') if item.strip()]
            return tuple(sorted(items))
        else:
            return ingredient

In [87]:
df['NLP_Ingredients'] = df['NLP_Ingredients'].apply(remove_empty)
df.head(10)

,ID,Name,Description,IngredientsExtracted,IngredientsRaw,serving_size,Servings,Instructions,Tags,SearchTerms,...,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,NLP_Ingredients
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","('water', 'grits', 'salt', 'cheddar cheese', '...","('4 cups water', '1 cup uncooked old ...",1 (155 g),8,"('I a sauce pan, bring water to a boil; slowly...","frozenset({'stove-top', 'main-ingredient', 'se...","('low-calorie', 'side', 'vegetarian', 'diabeti...",...,144.8,6.7,3.3,14.9,382.7,15.7,0.9,0.2,5.0,"(cheddar cheese, garlic, grit, olive oil, salt..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","('onion', 'red bell pepper', 'garlic cloves', ...","('1 medium onion, chopped coarse ', '1 ...",1 (366 g),4,"('In a food processor, pulse the onion, red pe...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner', 'shrimp')",...,756.5,28.9,9.0,191.8,2094.9,82.3,3.9,9.7,39.2,"(andouille sausage, bay leaf, clam juice, fres..."
2,41090,black-and-white bean salad,NaN,"('white beans', 'canned black beans', 'tomatoe...","('1 cup canned white beans, rinsed and dra...",1 (807 g),1,"('In a large bowl, combine beans, tomato, onio...","frozenset({'technique', 'main-ingredient', 'pr...","('salad', 'vegan', 'side', 'vegetarian', 'dinn...",...,159.0,1.6,0.3,0.0,318.3,28.3,8.3,2.1,9.1,"(black bean, black pepper, celery, italian par..."
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"('zucchini', 'yellow squash', 'diced tomatoes'...","('2 zucchini, sliced ', '2 small ye...",1 (244 g),4,('Put all ingredients in the crock pot and coo...,"frozenset({'beginner-cook', 'main-ingredient',...","('side', 'vegetarian', 'italian')",...,47.1,0.4,0.1,0.0,19.1,10.4,2.7,4.6,2.6,"(garlic, green bell pepper, italian seasoning,..."
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"('beef stew meat', 'flour', 'salt', 'allspice'...","('3 lbs beef stew meat', '3 tablespoons...",1 (358 g),8,"('Preheat oven to 350°F.', ""Cut beef into 1 in...","frozenset({'main-ingredient', 'course', 'prepa...","('dinner',)",...,691.1,45.8,17.7,177.1,503.6,14.4,1.6,6.8,48.2,"(allspice, beef stew meat, beef stock, black p..."
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"('slivered almonds', 'cider vinegar', 'sugar',...","('12 ounces slivered almonds', '1/4 cup ...",1 (832 g),1,('Preheat oven to 375°F Place almonds in sing...,"frozenset({'course', 'preparation', 'candy', '...","('dessert',)",...,2051.9,96.9,7.3,0.0,2668.3,280.2,24.6,243.1,41.9,"(cayenne pepper, cider vinegar, ground coriand..."
6,232076,Retro Chicken & Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","('chicken breasts', 'green onion', 'red bell p...","('4 cups roasted chopped chicken breasts',...",1 (85 g),6,"('In large bowl, combine chicken, green onion,...","frozenset({'30-minutes-or-less', 'main-ingredi...","('casserole', 'dinner', 'chicken')",...,515.5,39.0,15.4,49.0,649.3,30.6,2.4,4.0,13.3,"(cheddar cheese, chicken breast, dijon mustard..."
7,232083,Asparagus Omelette Wraps,"These wraps make a lovely breakfast, light lun...","('eggs', 'milk', 'fresh sage', 'fresh thyme', ...","('8 eggs', '1/2 cup milk', '1 tabl...",1 (499 g),4,"('Beat the eggs in a bowl. Add the milk, sage,...","frozenset({'cuisine', 'eggs', 'asparagus', 'ma...","('gluten-free', 'breakfast')",...,305.4,18.2,5.0,376.3,207.8,17.8,7.5,5.1,22.4,"(asparagus, egg, extra virgin olive oil, fresh..."
8,79222,Potato-Crab Chowder,Soup for the soul!,"('butter', 'onion', 'garlic', 'potatoes', 'flo...","('2 tablespoons butter', '1 medium o...",1 (362 g),6,('Saute onion& garlic in melted butter in larg...,"frozenset({'healthy', 'stove-top', 'main-ingre...","('low-calorie', 'healthy', 'low-sodium', 'low-...",...,274.2,8

In [88]:

df.to_csv('recipes_food_com_revised.csv')